#Setting up

In [ ]:
!pip install diffusers transformers accelerate

In [ ]:
!accelerate config default

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import UniPCMultistepScheduler
from diffusers import StableDiffusionControlNetPipeline
from diffusers.utils import load_image
import PIL
import torch
import io
from google.colab import files

# Loading Model

In [ ]:
#@title **Hugging Face model**

model_repository = 'GreeneryScenery/SheepsControlV3' #@param {type:"string"}
stable_diffusion_repository = 'stabilityai/stable-diffusion-2-1-base' #@param {type:"string"}

controlnet = ControlNetModel.from_pretrained(model_repository, torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    stable_diffusion_repository, controlnet=controlnet, torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# this command loads the individual model components on GPU on-demand.
pipe.enable_model_cpu_offload()

#Generate image

In [ ]:
#@title **Calling model**

seed = 1 #@param {type:"slider", min:0, max:10000, step:1}
prompt = "Sheep" #@param {type:"string"}
# , trending on artstation, artstationHD, artstationHQ, patreon, 4k, 8k

generator = torch.manual_seed(seed)

upload = True #@param {type:"boolean"}

if not upload:
  #@markdown  Link and filename are ignored if uploading file
  filename = '1808816.png' #@param {type:"string"}
  repository = 'GreeneryScenery/SheepsControlV3' #@param {type:"string"}
  link = f'https://huggingface.co/{repository}/resolve/main/{filename}' #@param {type:"string"}
  !wget $link
  init_image = PIL.Image.open(filename).convert("RGB")
else:
  uploaded = files.upload()
  init_image = PIL.Image.open(io.BytesIO(uploaded[next(iter(uploaded))])).convert("RGB")

init_image.thumbnail((256, 256))

out_image = pipe(
    prompt, num_inference_steps=20, generator=generator, image=init_image
).images[0]

init_image.show()
out_image.show()

out_image.save(f"{prompt}.png")